In [ ]:
import polars as pl
import numpy as np
from pathlib import Path
from polimi.utils._custom import load_history, load_articles, load_behaviors

In [ ]:
base_path = Path('../../dataset')
type = 'demo'
split = 'train'
CSV_STORAGE = Path('dataset/graph').joinpath(type)

a = load_articles(base_path, type, split)
h = load_history(base_path, type, split)
b = load_behaviors(base_path, type, split)

# Fix

In [ ]:
a = a.with_columns(pl.when(pl.col('category_str') == 'bibliotek').then(2504).otherwise(pl.col('category')).alias('category'))
a = a.with_columns(pl.when(pl.col('category') == 68).then(pl.lit("video")).otherwise(pl.col('category_str')).alias('category_str'))

#creation homepage node
home_page_article_dict = {k: None for k in a.columns}
home_page_article_id = 0
home_page_article_dict['article_id'] = home_page_article_id
home_page_article_dict['title'] = 'homepage'
home_page_article_dict['article_type'] = 'homepage'
a = a.vstack(pl.DataFrame(home_page_article_dict, schema=a.schema))

#link null article id in impression to homepage node
b = b.with_columns(pl.col('article_id').fill_null(home_page_article_id))


## Creation Neo4j CSV

In [ ]:
b_tmp = b.select('user_id', 'gender', 'postcode', 'age').group_by('user_id').first()
u_data = h.select('user_id').join(b_tmp, on='user_id', how='left').sort('user_id').unique('user_id')
u_data = u_data.with_columns(pl.lit('User').alias(':LABEL'))
u_data=u_data.rename({'user_id': 'id:ID(User)'})

u_data.write_csv(CSV_STORAGE / 'user_data.csv')
u_data.head(3)

In [ ]:
i_data = b.drop('article_id', 'article_ids_inview','article_ids_clicked', 'user_id',
                'gender', 'postcode', 'age')\
          .sort('impression_id')


i_data = i_data.with_columns(pl.lit('Impression').alias(':LABEL'))
i_data=i_data.rename({'impression_id': 'id:ID(Impression)'})
i_data.write_csv(CSV_STORAGE / 'impression_data.csv', datetime_format='%Y-%m-%dT%H:%M:%S')
i_data.head(3)

In [ ]:
topic = a.select(pl.col('topics').alias('topic')).explode('topic').unique().sort('topic').drop_nulls()
topic = topic.insert_column(0, pl.Series("topic_id", np.arange(topic.shape[0])))
topic = topic.with_columns(pl.lit('Topic').alias(':LABEL'))
topic=topic.rename({'topic': 'name', 'topic_id': 'id:ID(Topic)'})
topic.write_csv(CSV_STORAGE / 'topic_data.csv')
topic.head(3)

In [ ]:
entity_group = a.select(pl.col('entity_groups').alias('entity_group')).explode('entity_group').unique().sort('entity_group').drop_nulls()
entity_group = entity_group.insert_column(0, pl.Series("entity_group_id", np.arange(entity_group.shape[0])))
entity_group = entity_group.with_columns(pl.lit('Entity').alias(':LABEL'))
entity_group = entity_group.rename({'entity_group': 'name', 'entity_group_id': 'id:ID(Entity)'})
entity_group.write_csv(CSV_STORAGE / 'entity_data.csv')
entity_group.head(3)

In [ ]:
cats = a.select(pl.col('category').alias('category_id'), pl.col('category_str').alias('name')).unique().drop_nulls()
ids_cats = cats.select('category_id').to_numpy().reshape(-1)
sub_cats = a.select(pl.col('subcategory').alias('category_id')).explode('category_id').unique().drop_nulls()
sub_cats = sub_cats.filter(~pl.col('category_id').is_in(cats.select('category_id'))).with_columns(pl.lit(None).alias('name'))
category = cats.vstack(sub_cats).sort('category_id')
category = category.with_columns(pl.lit('Category').alias(':LABEL')).sort('category_id').unique()
category=category.rename({'category_id': 'id:ID(Category)'})
category.write_csv(CSV_STORAGE / 'category_data.csv')
category.head(3)

In [ ]:
a_to_category = a.select('article_id', pl.lit(False).alias('is_sub'), pl.col('category').alias('category_id')).unique().drop_nulls()
a_to_sub_category = a.select('article_id', pl.lit(True).alias('is_sub'), pl.col('subcategory').alias('category_id')).explode('category_id').unique().drop_nulls()

a_to_category = a_to_category.vstack(a_to_sub_category).rename({'article_id': ':START_ID(Article)', 'category_id': ':END_ID(Category)'})\
    .with_columns(pl.lit('IS_IN').alias(':TYPE'))
a_to_category.write_csv(CSV_STORAGE / 'article_to_category.csv')
a_to_category.head(3)

In [ ]:
a_to_topic = a.select('article_id', pl.col('topics').alias('name')).explode('name').sort('article_id').unique().drop_nulls()
a_to_topic = a_to_topic.join(topic, on='name', how='left').select('article_id', pl.col('id:ID(Topic)').alias('topic_id'))\
    .with_columns(pl.lit('IS_IN').alias(':TYPE'))
a_to_topic = a_to_topic.rename({'article_id': ':START_ID(Article)', 'topic_id': ':END_ID(Topic)'})
a_to_topic.write_csv(CSV_STORAGE / 'article_to_topic.csv')
a_to_topic.head(3)

In [ ]:
a_to_entity = a.select('article_id', pl.col('entity_groups').alias('name')).explode('name').sort('article_id').unique().drop_nulls()
a_to_entity = a_to_entity.join(entity_group, on='name', how='left').select('article_id', pl.col('id:ID(Entity)').alias('entity_id'))\
        .with_columns(pl.lit('IS_IN').alias(':TYPE'))

a_to_entity = a_to_entity.rename({'article_id': ':START_ID(Article)', 'entity_id': ':END_ID(Entity)'})
a_to_entity.write_csv(CSV_STORAGE / 'article_to_entity.csv')
a_to_entity.head(3)

In [ ]:
a_data = a.drop('image_ids', 'ner_clusters', 'entity_groups', 'topics', 'subcategory', 
                'category', 'category_str')\
                    .with_columns(pl.col('title').str.replace_all('\n', ''))\
                    .with_columns(pl.col('subtitle').str.replace_all('\n', ''))\
                    .with_columns(pl.col('body').str.replace_all('\n', ''))\
                    .with_columns(pl.lit('Article').alias(':LABEL'))\
                    .sort('article_id')

a_data = a_data.rename({'article_id': 'id:ID(Article)'})
a_data.write_csv(CSV_STORAGE / 'article_data.csv', datetime_format='%Y-%m-%dT%H:%M:%S')
a_data.head(3)

In [ ]:
u_to_i = b.select('user_id', 'impression_id') \
    .sort('user_id').unique().drop_nulls() \
    .with_columns(pl.lit('GENERATE').alias(':TYPE'))
    
u_to_i = u_to_i.rename({'user_id': ':START_ID(User)', 'impression_id': ':END_ID(Impression)'})
u_to_i.write_csv(CSV_STORAGE / 'user_to_impression.csv')
u_to_i.head(3)

In [ ]:
i_to_a = b.select('impression_id', pl.col('article_id')).sort('impression_id').unique().drop_nulls()\
    .with_columns(pl.lit('ON').alias(':TYPE'))
i_to_a = i_to_a.rename({'impression_id': ':START_ID(Impression)', 'article_id': ':END_ID(Article)'})
i_to_a.write_csv(CSV_STORAGE / 'impression_to_article.csv')
i_to_a.head(3)

In [ ]:
i_to_a_clicked = b.select('impression_id', pl.col('article_ids_clicked').alias('article_id')).explode('article_id').sort('impression_id')\
    .with_columns(pl.lit('CLICK').alias(':TYPE'))
i_to_a_clicked = i_to_a_clicked.rename({'impression_id': ':START_ID(Impression)', 'article_id': ':END_ID(Article)'})
i_to_a_clicked.write_csv(CSV_STORAGE / 'impression_to_article_clicked.csv')
i_to_a_clicked.head(3)

In [ ]:
i_to_a_inview = b.select('impression_id', pl.col('article_ids_inview').alias('article_id')).explode('article_id').sort('impression_id').drop_nulls()
i_to_a_inview = i_to_a_inview.rename({'impression_id': ':START_ID(Impression)', 'article_id': ':END_ID(Article)'})\
    .with_columns(pl.lit('INVIEW').alias(':TYPE'))
i_to_a_inview.write_csv(CSV_STORAGE / 'impression_to_article_inview.csv')
i_to_a_inview.head(3)

### History

In [ ]:
batch = 0
batch_size=100
user_ids = h.select('user_id')

h_final = None
for batch in range(0, len(user_ids), batch_size):
    
    user_ids_batch = user_ids[batch:batch+batch_size]
    h_data = h.filter(pl.col('user_id').is_in(user_ids[batch:batch+batch_size]))\
    .explode('impression_time_fixed', 'article_id_fixed', 'scroll_percentage_fixed', 'read_time_fixed')

    h_data = h_data.with_columns(pl.Series(np.arange(-1, len(h_data) - 1)).cast(pl.Int32).alias('id_l'))
    h_data = h_data.with_columns(pl.Series(np.arange(len(h_data))).cast(pl.Int32).alias('id_r'))
    h_data = h_data.join(h_data, left_on=['user_id', 'id_r'], right_on=['user_id', 'id_l'], how='left')

    h_data = h_data.select(
        pl.col('article_id_fixed').alias(':START_ID(Article)'),
        'user_id', 
        pl.col('impression_time_fixed').alias('impression_time'),
        pl.col('scroll_percentage_fixed').alias('scroll_percentage'),
        pl.col('read_time_fixed').alias('read_time'),
        pl.col('article_id_fixed_right').alias(':END_ID(Article)'),
        pl.lit('FOLLOW').alias(':TYPE')
    ).drop_nulls(':END_ID(Article)')
    
    if h_final is None:
        h_final = h_data
    else:
        h_final = h_final.vstack(h_data)
    

h_final.write_csv(CSV_STORAGE / 'history_article_to_article.csv', datetime_format='%Y-%m-%dT%H:%M:%S')
h_final.head(3)

In [ ]:
h_data = h.select(pl.col('user_id').alias(':START_ID(User)'), 
                  pl.col('impression_time_fixed').list.first().alias('impression_time'),
                  pl.col('scroll_percentage_fixed').list.first().alias('scroll_percentage'),
                  pl.col('read_time_fixed').list.first().alias('read_time'),
                  pl.col('article_id_fixed').list.first().alias(':END_ID(Article)'),
                  pl.lit('FOLLOW').alias(':TYPE'))

h_data.write_csv(CSV_STORAGE / 'history_user_to_article.csv', datetime_format='%Y-%m-%dT%H:%M:%S')
h_data.head(3)

In [ ]:
' '.join(
    [
        'bin/neo4j-admin database import full',
        #nodes
        f'--nodes=User=import/{type}/user_data.csv',
        f'--nodes=Article=import/{type}/article_data.csv',
        #relationships
        f'--relationships=import/{type}/history_user_to_article.csv',
        f'--relationships=import/{type}/history_article_to_article.csv',
        
        '--overwrite-destination',
        f'newsdb-history-{type}'
        ])

### Load neo4j-admin-script

In [ ]:
from polimi.utils._graph import create_neo4j_admin_script

create_neo4j_admin_script('large')

## Contraints

In [ ]:
from neo4j import GraphDatabase
from polimi.utils._graph import add_constraint
from dotenv import load_dotenv
import os

load_dotenv()

# Retrieve Neo4j connection details
URI = os.getenv("NEO4J_URI")
USER = os.getenv("NEO4J_AUTH_USER")
PASSW = os.getenv("NEO4J_AUTH_PASSWORD")

# Establish connection
driver = GraphDatabase.driver(URI, auth=(USER, PASSW), database=f'newsdb-{type}')

print(driver.verify_connectivity())
add_constraint(driver, 'Article', 'id')
add_constraint(driver, 'Impression', 'id')
add_constraint(driver, 'User', 'id')
add_constraint(driver, 'Topic', 'id')
add_constraint(driver, 'Category', 'id')
add_constraint(driver, 'Entity', 'id')


driver.close()